In [ ]:
# Updated code, now including the addition of the "Momentum Health Indicator" column to calculate the difference 
# between the percentage of days the stock went up and the percentage of days it went down over the past year

# Starting code to import yearly returns and P/E ratios for momentum investing strategy

import yfinance as yf
import pandas as pd

def get_sp500_tickers():
    table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    tickers = table[0]['Symbol'].tolist()
    return tickers

def fetch_and_analyze_data(ticker):
    try:
        adjusted_ticker = ticker.replace('.', '-')
        stock_data = yf.Ticker(adjusted_ticker).history(start='2023-12-29', end='2024-12-31')

        if not stock_data.empty:
            stock_data.index = stock_data.index.tz_localize(None)
            
            start_price = stock_data.iloc[0]['Close']
            end_price = stock_data.iloc[-1]['Close']
            annual_return = (end_price - start_price) / start_price * 100
            
            stock_data['Daily Return'] = stock_data['Close'].pct_change()
            std_dev = stock_data['Daily Return'].std() * (252**0.5)
            sharpe_ratio = (annual_return / 100 - risk_free_rate) / std_dev
            
            # Fetch P/E ratio using yfinance
            pe_ratio = yf.Ticker(adjusted_ticker).info.get('trailingPE', None)

            # Calculate Momentum Health Indicator
            stock_data['up_days'] = stock_data['Daily Return'] > 0
            stock_data['down_days'] = stock_data['Daily Return'] < 0

            up_days_percentage = stock_data['up_days'].sum() / len(stock_data) * 100
            down_days_percentage = stock_data['down_days'].sum() / len(stock_data) * 100

            stock_data['Momentum Health Indicator'] = up_days_percentage - down_days_percentage

            # Drop the temporary columns
            stock_data.drop(columns=['up_days', 'down_days'], inplace=True)
            
            return {'Ticker': ticker, '2024 Return': annual_return, '2024 Std Dev': std_dev, 'Sharpe Ratio': sharpe_ratio, 'P/E Ratio': pe_ratio, 'Momentum Health Indicator': stock_data['Momentum Health Indicator'].iloc[-1]}, stock_data
        else:
            return None, None
    except Exception as e:
        print(f"Skipping {ticker} due to error: {e}")
        return None, None

sp500_tickers = get_sp500_tickers()
returns_data = []
risk_free_rate = 0.02


writer = pd.ExcelWriter('sp500_data.xlsx', engine='openpyxl')

for ticker in sp500_tickers:
    result, stock_data = fetch_and_analyze_data(ticker)
    if result and stock_data is not None:
        returns_data.append(result)
        stock_data.to_excel(writer, sheet_name=result['Ticker'])

returns_df = pd.DataFrame(returns_data)
returns_df_sorted = returns_df.sort_values(by='2024 Return', ascending=False)
returns_df_sorted.to_excel(writer, sheet_name='Output', index=False)


writer.close()

print(returns_df_sorted[['Ticker', '2024 Return', '2024 Std Dev', 'Sharpe Ratio', 'P/E Ratio', 'Momentum Health Indicator']])


